<h1>Import Required Libraries</h1>


In [1]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# k-fold cross validation evaluation of XGBoost model
from numpy import loadtxt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
# plot decision tree
from xgboost import plot_tree
from matplotlib import pyplot
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

<h1> Dataset3 </h1>

In [2]:
train = pd.read_csv('train.csv')


display(train.head(3))


,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,13829,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown,no
1,22677,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown,no
2,10541,30,management,single,secondary,no,135,no,no,cellular,14,aug,2,-1,0,unknown,no


In [3]:
y_train = train['y']
x_train = train.drop(columns = ['ID','y'])


In [4]:
y_train = y_train.replace('no', 0).replace('yes', 1)

In [5]:
print(x_train.info())

print(x_train.shape)

print(x_train.isna().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12870 entries, 0 to 12869
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        12870 non-null  int64 
 1   job        12870 non-null  object
 2   marital    12870 non-null  object
 3   education  12870 non-null  object
 4   default    12870 non-null  object
 5   balance    12870 non-null  int64 
 6   housing    12870 non-null  object
 7   loan       12870 non-null  object
 8   contact    12870 non-null  object
 9   day        12870 non-null  int64 
 10  month      12870 non-null  object
 11  campaign   12870 non-null  int64 
 12  pdays      12870 non-null  int64 
 13  previous   12870 non-null  int64 
 14  poutcome   12870 non-null  object
dtypes: int64(6), object(9)
memory usage: 1.5+ MB
None
(12870, 15)
age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month      

In [6]:
def encode(dataframe):
    lec = LabelEncoder()
    for j in dataframe.columns:
        if(dataframe[j].dtype == 'object'):
            dataframe[j] = lec.fit_transform(dataframe[j])

encode(train)


In [7]:
x_train = pd.get_dummies(x_train, drop_first = True)


In [8]:
x_train.describe()

,age,balance,day,campaign,pdays,previous,job_blue-collar,job_entrepreneur,job_housemaid,job_management,...,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_other,poutcome_success,poutcome_unknown
count,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,...,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000,12870.000000
mean,41.091142,1483.774437,15.641103,2.659130,45.555478,0.688967,0.199767,0.029759,0.025952,0.222067,...,0.141647,0.115307,0.018415,0.279254,0.085004,0.026263,0.021290,0.044678,0.065657,0.782440
std,11.305560,3311.055181,8.368983,2.863507,104.449411,2.049696,0.399841,0.169929,0.158998,0.415652,...,0.348702,0.319404,0.134452,0.448650,0.278898,0.159922,0.144354,0.206603,0.247690,0.412603
min,18.000000,-6847.000000,1.000000,1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32.000000,102.000000,8.000000,1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,39.000000,515.000000,16.000000,2.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,49.000000,1591.750000,21.000000,3.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,95.000000,102127.000000,31.000000,43.000000,854.000000,58.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(x_train,y_train, test_size=0.3, random_state=1)

<h2> 1. XGBoost Classifier</h2?

<h3> Sử dụng train-test set</h3>

In [10]:
model1 = XGBClassifier()
model1.fit(X_train, Y_train)
predictions = model1.predict(X_test)
accuracy = accuracy_score(Y_test, predictions)
print('Accuracy: %.2f%%' % (accuracy*100))

Accuracy: 78.53%


<h3>k-fold cross-validation</h3>

In [13]:
model1 = XGBClassifier()
kfold = KFold(n_splits=10,shuffle=True, random_state=7)
results = cross_val_score(model1, x_train, y_train, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 78.62% (1.20%)


<h2> 2. Gradient Boosting Classifier </h2>

<h3> Sử dụng train-test set</h3>

In [12]:
model2 = GradientBoostingClassifier()
model2.fit(X_train, Y_train)
predictions = model2.predict(X_test)
accuracy = accuracy_score(Y_test, predictions)
print('Accuracy: %.2f%%' % (accuracy*100))

Accuracy: 78.09%


<h3>k-fold cross-validation</h3>

In [14]:
# CV model
model2 = GradientBoostingClassifier()
kfold = KFold(n_splits=10,shuffle=True, random_state=7)
results = cross_val_score(model2, x_train, y_train, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 78.51% (1.10%)


<h2> 3. Random Forest Classifier </h2>

<h3> Sử dụng train-test set</h3>

In [15]:
model3 = RandomForestClassifier()
model3.fit(X_train, Y_train)
predictions = model3.predict(X_test)
accuracy = accuracy_score(Y_test, predictions)
print('Accuracy: %.2f%%' % (accuracy*100))

Accuracy: 78.06%


k-fold cross-validation

In [17]:
model3 = RandomForestClassifier()
kfold = KFold(n_splits=10,shuffle=True, random_state=7)
results = cross_val_score(model3, x_train, y_train, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 78.23% (1.29%)
